In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import os
import json
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY1"))


In [2]:
def call_llm(prompt: str, system:str = None,model: str = "gpt-4o-mini", temperature: float = 0.7) -> str:
    try:
        messages = []
        if system:
            messages = [{"role":"system","content":system}]
        messages.append({"role": "user", "content": prompt})
        response = client.chat.completions.create(
            model="gpt-4",
            messages=messages
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error in call_llm func: {e}"

In [3]:
def extract_json_dict(text: str):
    try:
        start = min(
            (text.index('{') if '{' in text else float('inf')),
            (text.index('[') if '[' in text else float('inf'))
        )
        end = max(
            (text.rindex('}') + 1 if '}' in text else -1),
            (text.rindex(']') + 1 if ']' in text else -1)
        )
        json_str = text[start:end]
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        print(text)
        print(json_str)
        raise ValueError(f"Invalid JSON found: {e}")

In [4]:
def extract_json_dict(text: str):
    try:
        start = min(
            (text.index('{') if '{' in text else float('inf')),
            (text.index('[') if '[' in text else float('inf'))
        )
        end = max(
            (text.rindex('}') + 1 if '}' in text else -1),
            (text.rindex(']') + 1 if ']' in text else -1)
        )
        json_str = text[start:end]
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        print(text)
        print(json_str)
        raise ValueError(f"Invalid JSON found: {e}")

In [ ]:
ml_resume = '''
{
  "experience": [
    {
      "company": "The Barabari Collective",
      "position": "AI Engineer (Freelance)",
      "duration": "June 2025 - Present"
    },
    {
      "company": "GDSC DDU Chapter",
      "position": "AI/ML Team Member",
      "duration": "2023-2024"
    }
  ],
  "certifications": [
    "Advanced Learning Algorithms - DeepLearning.AI",
    "Introduction to TensorFlow for AI, ML and DL - DeepLearning.AI",
    "Supervised Machine Learning - Stanford University"
  ],
  "projects": [
    {
      "name": "ChatGPT 2",
      "description": "Trained and implemented from scratch in Pytorch. Winner of Bhashathon 2025, won a cash prize of Rs. 50,000."
    }
  ],
  "skills": {
    "Languages": [
      "Python",
      "C++",
      "JavaScript"
    ],
    "Libraries": [
      "PyTorch",
      "NumPy",
      "Pandas",
      "Matplotlib",
      "FAISS",
      "scikit-learn"
    ],
    "Frameworks": [
      "Flask",
      "FastAPI",
      "Express.js",
      "TensorFlow"
    ],
    "Tools & Technologies": [
      "Git",
      "AWS",
      "Linux",
      "SentencePiece",
      "OpenAI",
      "OpenAI-Agents",
      "Deepgram",
      "openSMILE"
    ]
  }
}
'''

In [6]:
# mention ratios in catagories of questions - 60-20-10-10
# add hints to each question


prompt_template = """
**ROLE**
You are an expert interviewer with deep experience in hiring for the role of {Role}. Your task is to create highly tailored, insightful interview questions based on the candidate's resume, work history, skills, and years of experience.

### INSTRUCTION
1. **Question Generation**
   - Create **15-20 questions** covering these categories, elaborate on questions to help the candidate understand the question better:
     - **Technical**: Core DS concepts (ML, stats, coding)
     - **Behavioral**: Soft skills, teamwork, problem-solving
     - **Role-specific**: Job description alignment (e.g., NLP, MLOps)
     - **Resume-specific**: Directly from resume content (projects, skills, gaps)
   - **Label each question** with its category (e.g., `[Technical]`).

2. **Difficulty Settings** (based on years of experience):
   | **Experience** | **Easy** | **Medium** | **Hard** | **Distribution** |
   |----------------|----------|------------|----------|-----------------------|
   | **0-2 years** | Foundational concepts | Basic applications | Theoretical depth | 50% Easy, 30% Medium, 20% Hard |
   | **2-5 years** | Applied scenarios | Optimization trade-offs | System design | 30% Easy, 40% Medium, 30% Hard |
   | **5+ years** | Edge cases | Architecture decisions | Leadership/strategy | 20% Easy, 30% Medium, 50% Hard |

   - **Difficulty Definitions**:
     - **Easy**: Definitions, basic syntax, simple scenarios.
     - **Medium**: Applied problem-solving, trade-off analysis.
     - **Hard**: System design, optimization, failure mitigation.

3. **Resume Integration**
   - Extract **3 resume-specific questions** targeting:
     - Projects listed
     - Skills claimed (e.g., "Python," "Express")
     - Experience gaps/job hops

4. **Output Format**
   ```markdown
   [Category] Difficulty: [Easy/Medium/Hard]
   Question: [Your question here]
   ```
   - **Do NOT** add explanations or numbering.
   
### EXAMPLE
{examples}

### CONTEXT (Knowledge Grounding / Syllabus)
Questions must draw exclusively from these topics:
{context}

### TASK
Generate questions for:
- **Years of Experience**: {years_of_experience}
- **Resume**: {user_resume}
{job_description}

**Output**: 15-20 questions following the format above.

### KEY RULES
1. **No open-ended decisions**: All questions MUST adhere to the syllabus and difficulty rules.
2. **Resume fidelity**: Resume-specific questions must reference exact projects/skills.
3. **Diversity**: Cover ≥2 topics per category (e.g., stats + ML for Technical).
4. **Role alignment**: 40% of questions must map to the job description.
"""


In [ ]:
# add examples of all difficulties
# https://akanksha-paul.medium.com/interview-experience-for-applied-scientist-at-amazon-5ff5003375dd

ML_examples = '''
**Candidate**: 3 years of experience, resume includes "CNN image classification project."
```markdown
[Technical] Difficulty: Medium
Question: Explain how you'd mitigate overfitting in a CNN for image classification.

[Behavioral] Difficulty: Easy
Question: Describe a time you resolved a conflict in a cross-functional team.

[Role-specific] Difficulty: Hard
Question: Design a real-time fraud detection system for transactional data (assume 1M RPM).

[Resume-specific] Difficulty: Medium
Question: Your resume mentions a CNN project—what data augmentation techniques did you use and why?
```
'''

ML_context = '''
Questions must draw **exclusively** from these topics:

### **Enhanced Technical Syllabus**  
**1. Statistics & Probability**  
- *Hypothesis Testing*: T-tests, Z-tests, ANOVA, p-values, Type I/II errors  
- *Bayesian Inference*: Priors/posteriors, Bayes' theorem applications  
- *Distributions*: Gaussian, Poisson, Binomial properties and use cases  
- *A/B Testing*: Power analysis, sequential testing, covariate adjustment  

**2. Machine Learning**  
- *Supervised Learning*:  
  - Algorithms: Linear/logistic regression, SVM, tree-based methods (RF, XGBoost)  
  - Evaluation: ROC curves, precision-recall tradeoffs, cross-validation strategies  
- *Unsupervised Learning*: K-means clustering, PCA, anomaly detection  
- *Regularization*: L1/L2 penalties, dropout, early stopping  

**3. Coding & Tools**  
- *Python*:  
  - Pandas (data wrangling, time-series manipulation)  
  - Scikit-learn (pipeline construction, hyperparameter tuning)  
- *SQL*: Window functions, query optimization, nested queries  
- *Git*: Branching strategies, rebase vs. merge, CI/CD integration  
- *Cloud Platforms*:  
  - AWS (SageMaker, Redshift) / GCP (BigQuery, Vertex AI)  

**4. Data Engineering**  
- *ETL Pipelines*: Batch vs. stream processing (Airflow vs. Kafka)  
- *Data Warehousing*: Star/snowflake schemas, slowly changing dimensions  
- *Big Data Tools*: Spark (RDD/DataFrame API), Hadoop ecosystem  

**5. Advanced Topics**  
- *Deep Learning*:  
  - CNNs (architectures like ResNet, transfer learning)  
  - RNNs (LSTM/GRU, sequence-to-sequence models)  
- *NLP*: Transformer architecture, fine-tuning BERT, attention mechanisms  
- *MLOps*: Model monitoring, drift detection, feature stores  

---

### **Behavioral Syllabus**  
**Core Framework**  
- **STAR Method**: Structured storytelling (Situation, Task, Action, Result) with quantifiable outcomes  
- **Prioritization**: Eisenhower matrix, ROI-based task triage, resource constraints  
- **Ethical Dilemmas**: Data privacy (GDPR/CCPA), model bias mitigation, explainability tradeoffs  
- **Stakeholder Communication**: Tailoring messages to executives vs. engineers, conflict resolution  

**Integrated Amazon Leadership Principles**  
1. **Earn Trust**:  
   - *Focus*: Building psychological safety, admitting mistakes, delivering on commitments  
   - *Sample Q*: "Describe a time you received critical feedback. How did you rebuild trust?"  

2. **Are Right, A Lot**:  
   - *Focus*: Data-driven decision-making, balancing intuition with evidence, handling ambiguous data  
   - *Sample Q*: "When did you advocate for a counterintuitive solution backed by data?"  

3. **Invent and Simplify**:  
   - *Focus*: Creating scalable solutions, reducing technical debt, elegant problem-solving  
   - *Sample Q*: "Share an example where you turned a complex process into a simple solution."  

### **Role-Specific Additions**  
**NLP Specialist**  
- *Must-Know*: Attention mechanisms, transformer variants (RoBERTa, T5), Hugging Face ecosystem  
- *Tools*: spaCy, NLTK, BERTopic  

**ML Engineer**  
- *Must-Know*: Containerization (Docker), serverless deployment, model versioning (MLflow)  
- *Tools*: Kubernetes, TF Serving, Prometheus for monitoring  

**Analytics Lead**  
- *Must-Know*: Causal inference (propensity scoring, DiD), cohort analysis, monetization metrics  
- *Tools*: Optimizely, Mixpanel, Monte Carlo simulations  
'''
# be more explicit on syllabus

# amazon leadership principles could be added in behavioral
# Earn Trust
# Are Right, A Lot
# Invent and Simplify


In [8]:
frontend_examples = '''
##Example 1
Candidate: 1 year of experience, resume includes "Mobile-first PWA for local business."

```markdown
[Technical] Difficulty: Easy  
Question: When would you use CSS Grid vs. Flexbox? Provide layout examples.  

[Behavioral] Difficulty: Medium  
Question: How would you handle disagreements with a designer about implementation feasibility?  

[Role-specific] Difficulty: Medium  
Question: Design a service worker caching strategy for a PWA with frequently updated product catalogs.  

[Resume-specific] Difficulty: Hard  
Question: Your PWA claims 40% faster TTI—what metrics did you track and how did you achieve this?
```

##Example 2
Candidate: 6 years of experience, resume includes "Micro-frontend architecture migration."

```markdown
[Technical] Difficulty: Hard  
Question: Compare hydration strategies for SSR applications (e.g., React vs Qwik).  

[Behavioral] Difficulty: Hard  
Question: Describe a technical debt situation you inherited and how you drove systemic fixes.  

[Role-specific] Difficulty: Medium  
Question: How would you implement a design system to ensure consistency across micro-frontends?  

[Resume-specific] Difficulty: Medium  
Question: Your migration project mentions Webpack Module Federation—what were the biggest integration challenges?
```
'''

frontend_context = """
#### **Technical Syllabus**  
1. **Core Web Technologies**  
   - HTML5 (semantic elements, accessibility), CSS3 (Flexbox/Grid, animations), JavaScript (ES6+, async/event loop)  
   - DOM manipulation, browser APIs, Web Performance (Critical Rendering Path, Lighthouse)  

2. **Frontend Frameworks & Libraries**  
   - React (hooks, state management), Vue, Angular, Svelte  
   - Component lifecycle, virtual DOM, SSR/SSG (Next.js, Nuxt)  

3. **State Management & Data Handling**  
   - Redux, Context API, React Query, GraphQL/Apollo  
   - RESTful APIs, WebSockets, error handling  

4. **Styling & Design Systems**  
   - CSS-in-JS (Styled Components, Emotion), preprocessors (Sass)  
   - Responsive design, cross-browser compatibility, UI/UX principles  

5. **Tooling & Workflow**  
   - Build tools (Webpack, Vite), testing (Jest, React Testing Library, Cypress)  
   - CI/CD pipelines, package managers (npm/yarn), TypeScript  

#### **Behavioral Syllabus**
   - STAR method, prioritization, ethical dilemmas, stakeholder communication

#### **Role-specific Syllabus**  
   - **UX-Focused Frontend**: Component libraries, interaction design  
   - **Performance Specialist**: Bundle optimization, Core Web Vitals  
   - **Full-Stack Frontend**: API integration, BFF (Backend For Frontend) patterns  
   - **Accessibility Engineer**: WCAG compliance, ARIA, assistive tech testing  
"""

In [9]:
backend_examples = '''
Candidate: 3 years backend experience, resume highlights "Scaled payment API handling 5K RPM"

##Example 1
```markdown
[Technical] Difficulty: Medium  
Question: How would you implement idempotency in a payment processing API?

[Behavioral] Difficulty: Medium  
Question: Tell me about a time you had to refactor critical code under tight deadlines. What trade-offs did you consider?

[Role-specific] Difficulty: Hard  
Question: Design a rate-limiting system for our API that supports 100K+ unique clients with dynamic throttling rules.

[Resume-specific] Difficulty: Medium  
Question: Your payment API handled 5K RPM - what strategies did you use for database connection pooling under load?
```

##Example 2
Candidate: 1 year experience, resume includes "Deployed serverless microservices on AWS"

```markdown
[Technical] Difficulty: Easy  
Question: Explain how you'd choose between SQS and Kafka for inter-service communication.

[Behavioral] Difficulty: Easy  
Question: Describe a technical challenge you faced during an internship and how you sought help.

[Role-specific] Difficulty: Medium  
Question: How would you design a file processing pipeline where uploads take >15 minutes? Consider cost and reliability.

[Resume-specific] Difficulty: Medium  
Question: For your serverless project, what monitoring metrics did you track and why?
```
'''


backend_context = '''
'''

In [ ]:
# - **Role**: 
ml_prompt = prompt_template.format(user_resume=ml_resume,
                                years_of_experience=0,
                                job_description="",
                                examples=ML_examples,
                                context=ML_context,
                                Role="Data Science",
                                )
frontend_prompt = prompt_template.format(user_resume=ml_resume,
                                years_of_experience=0,
                                job_description="",
                                examples=frontend_examples,
                                context=frontend_context,
                                Role="Frontend Developer",
                                )

backend_prompt = prompt_template.format(user_resume=ml_resume,
                                years_of_experience=0,
                                job_description="",
                                examples=backend_examples,
                                context=backend_context,
                                Role="backend Developer",
                                )


KeyError: 'years_of_experience'

In [30]:
res = call_llm(prompt=ml_prompt)

In [31]:
print(res)

[Technical] Difficulty: Easy
Question: Can you explain the difference between a list and a dictionary in Python?

[Technical] Difficulty: Medium
Question: Describe a scenario where you would use L1 regularization over L2 regularization.

[Technical] Difficulty: Hard
Question: How would you approach designing an A/B test to evaluate a new feature for a mobile application?

[Behavioral] Difficulty: Easy
Question: Can you describe a time when you had to collaborate with a team to solve a technical challenge?

[Behavioral] Difficulty: Medium
Question: When you've disagreed with your team on the direction of a project, how did you handle the situation?

[Role-specific] Difficulty: Medium
Question: Given your use of PyTorch for the ChatGPT 2 project, can you describe how PyTorch compares to TensorFlow for NLP tasks?

[Role-specific] Difficulty: Hard
Question: How would you manage deployment and testing while developing AI models in a ML Engineer role? Specifically, discuss any CI/CD workflow

In [1]:
a = {
  "feedback": "Quantitative Feedback:\nWords Per Minute (WPM): Your average pace: 108.1 WPM\nBenchmarking: Aim for 120-150 WPM in interviews\n\nPace Range Classification:\n- Too Slow: Your pace was slow 25.2% of the time\n- Ideal: You spoke at ideal pace for 47.7% of the time\n- Too Fast: Your pace exceeded 170 WPM for 0.9% of the time\n\nDetailed Pace Segments:\n\nToo slow segments:\n- [00:02 - 00:04]: so um one\n- [00:05 - 00:06]: was like\n- [00:24 - 00:32]: all and um it had just wasn't working good so um we\n- [01:00 - 01:06]: you know what I mean and so um yeah so\n- [01:07 - 01:12]: the model itself was like tough it's\n- [01:21 - 01:22]: tha and\n- [01:28 - 01:33]: accumulation and stuff and processed data in\n\nIdeal segments:\n- [00:07 - 00:18]: ODIA language you know it had like way less data than others so model was like kind of struggling to learn it properly I'm\n- [00:19 - 00:23]: you know the loss was and all\n- [00:33 - 00:34]: we tried to\n- [00:35 - 00:37]: like fix that by making batches\n- [00:39 - 00:42]: like you know make sure each batch\n- [00:46 - 00:50]: so like even if ODIA ka data kam tha\n- [00:51 - 00:53]: still came in training I\n- [00:54 - 01:00]: it helped kind of but not fully cause data hi kam tha\n- [01:18 - 01:21]: memory ka kaafi issue de raha tha\n- [01:23 - 01:24]: be limited tha\n- [01:25 - 01:27]: so um we did\n- [01:34 - 01:38]: like um small parts so it doesn't\n- [01:41 - 01:51]: like haan it was kind of difficult but I mean we did jugaad and somehow trained it and yeah I\n\nToo fast segments:\n- [01:51 - 01:57]: I learned a lot but still like next time we can do better you know what I mean\n"
}
print(a["feedback"])

Quantitative Feedback:
Words Per Minute (WPM): Your average pace: 108.1 WPM
Benchmarking: Aim for 120-150 WPM in interviews

Pace Range Classification:
- Too Slow: Your pace was slow 25.2% of the time
- Ideal: You spoke at ideal pace for 47.7% of the time
- Too Fast: Your pace exceeded 170 WPM for 0.9% of the time

Detailed Pace Segments:

Too slow segments:
- [00:02 - 00:04]: so um one
- [00:05 - 00:06]: was like
- [00:24 - 00:32]: all and um it had just wasn't working good so um we
- [01:00 - 01:06]: you know what I mean and so um yeah so
- [01:07 - 01:12]: the model itself was like tough it's
- [01:21 - 01:22]: tha and
- [01:28 - 01:33]: accumulation and stuff and processed data in

Ideal segments:
- [00:07 - 00:18]: ODIA language you know it had like way less data than others so model was like kind of struggling to learn it properly I'm
- [00:19 - 00:23]: you know the loss was and all
- [00:33 - 00:34]: we tried to
- [00:35 - 00:37]: like fix that by making batches
- [00:39 - 00:42